In [1]:
import findspark
findspark.init()
findspark.find()

'c:\\Users\\user\\miniconda3\\lib\\site-packages\\pyspark'

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, col, lit
from datetime import datetime
import logging
import json

In [2]:
spark = SparkSession.builder\
        .appName("silver_to_stgaging")\
        .enableHiveSupport()\
        .getOrCreate()

you will need to create a password.json file inorder to use the below code
file content example

{

    "password": "example@123"
    
}

In [25]:

with open('password.json', 'r') as pwd:
    data = json.load(pwd)
password = data['password']

In [13]:
list_of_tables = ['gfs_customers', 'gfs_discounts', 'gfs_employees', 'gfs_products', 'gfs_stores', 'gfs_transactions']

In [ ]:
# PostgreSQL connection details
pg_url = "jdbc:postgresql://localhost:5432/datawarehouse"
pg_properties = {
    "user": "postgres",
    "password": password,
    "driver": "org.postgresql.Driver"
}

In [22]:

def hive_to_stage(pg_properties, table):
# Table name in PostgreSQL

    df = spark.sql(f"SELECT * FROM {table}")
    pg_table = "stage.t_{}".format(table)

    # Write data to PostgreSQL, automatically creating the table
    df.write \
        .format("jdbc") \
        .option("url", pg_url) \
        .option("dbtable", pg_table) \
        .option("user", pg_properties["user"]) \
        .option("password", pg_properties["password"]) \
        .option("driver", pg_properties["driver"]) \
        .mode("overwrite") \
        .save()


In [24]:
for table in list_of_tables:
    hive_to_stage(pg_properties, table)

In [26]:
spark.stop()